# Data Preprocessing

Includes imputation and deletion of bad features.

### Import Dependencies

In [448]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

### Load Data

In [449]:
train_base_df = pd.read_csv("project/data/train/features.csv", parse_dates = ['date_recorded' ],  na_values = [0, '0'])
labels = pd.read_csv("project/data/train/labels.csv")

test_df = pd.read_csv("project/data/test/test.csv", parse_dates = ['date_recorded' ],  na_values = [0, '0'])

# Merge features and labels files
train_df = pd.merge(labels, train_base_df, on='id')

print("Numerical columns: \n", train_df.select_dtypes(include=np.number).columns.tolist())

# Introducing: DATA LEAKAGE
train_df = pd.concat([train_df, test_df])

Numerical columns: 
 ['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']


In [450]:
train_df.head(5)

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,NaN,Lake Nyasa,Mnyusi B,Iringa,11,5.0,Ludewa,Mundindi,109.0,True,GeoData Consultants Ltd,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,NaN,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,NaN,Lake Victoria,Nyamara,Mara,20,2.0,Serengeti,Natta,280.0,NaN,GeoData Consultants Ltd,Other,NaN,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,NaN,Pangani,Majengo,Manyara,21,4.0,Simanjiro,Ngorika,250.0,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,NaN,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,NaN,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63.0,Nanyumbu,Nanyumbu,58.0,True,GeoData Consultants Ltd,VWC,NaN,True,1986.0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,NaN,2011-07-13,Action In A,NaN,Artisan,31.130847,-1.825359,Shuleni,NaN,Lake Victoria,Kyanyamisa,Kagera,18,1.0,Karagwe,Nyakasimbi,NaN,True,GeoData Consultants Ltd,NaN,NaN,True,NaN,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


## 1. Data Imputation

In [451]:
missing = round((train_df.isna().sum())/len(train_df)*100,1)
missing = missing.sort_values(ascending = False)
missing.head(15)

num_private          98.7
amount_tsh           70.1
scheme_name          47.5
population           36.1
construction_year    35.0
gps_height           34.5
status_group         20.0
installer             7.4
funder                7.4
scheme_management     6.5
public_meeting        5.6
permit                5.1
longitude             3.1
subvillage            0.6
payment_type          0.0
dtype: float64

### Latitude and Longitude Imputation
0 values are replaced by subvillage, ward, lga, or region's mean logitude and latitude values for each column. 

Solution by BrendaLoznik

In [452]:
# train_df['longitude'].replace(0, np.nan, inplace=True)
#create mean longitude on the lowest granularity level (subvillage)
means_longitude_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['longitude'].mean().reset_index()
means_longitude_subvillage = means_longitude_subvillage.rename(columns={"longitude": "longitude_imputed_subvillage"})

#ward level
means_longitude_ward = train_df.groupby(['region', 'lga', 'ward',])['longitude'].mean().reset_index()
means_longitude_ward = means_longitude_ward.rename(columns={"longitude": "longitude_imputed_ward"})

#lga level
means_longitude_lga = train_df.groupby(['region', 'lga'])['longitude'].mean().reset_index()
means_longitude_lga = means_longitude_lga .rename(columns={"longitude": "longitude_imputed_lga"})

#region level
means_longitude_region = train_df.groupby(['region'])['longitude'].mean().reset_index()
means_longitude_region = means_longitude_region.rename(columns={"longitude": "longitude_imputed_region"})
means_longitude_region.head()

,region,longitude_imputed_region
0,Arusha,36.554067
1,Dar es Salaam,39.212935
2,Dodoma,36.041964
3,Iringa,34.895921
4,Kagera,31.233092


In [453]:
#merge the aggregated dataframes as new columns to the original df
train_df= train_df.merge(means_longitude_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
train_df= train_df.merge(means_longitude_ward, how = 'left', on = ['region', 'lga', 'ward'])
train_df = train_df.merge(means_longitude_lga, how = 'left', on = ['region', 'lga'])
train_df = train_df.merge(means_longitude_region, how = 'left', on = ['region'])

#select the right longitude level based on the availability of information
train_df['imputed_longitude'] = np.where(train_df['longitude'].isna(), train_df['longitude_imputed_subvillage'], train_df['longitude']) #if longitude is missing, impute it by the mean of the subvillage
train_df['imputed_longitude'] = np.where(train_df['imputed_longitude'].isna(), train_df['longitude_imputed_ward'], train_df['imputed_longitude']) #if subvillage mean is missing, impute it by the ward
train_df['imputed_longitude'] = np.where(train_df['imputed_longitude'].isna(), train_df['longitude_imputed_lga'], train_df['imputed_longitude'])
train_df['imputed_longitude'] = np.where(train_df['imputed_longitude'].isna(), train_df['longitude_imputed_region'], train_df['imputed_longitude'])

#drop redundant columns
train_df= train_df.drop(['longitude_imputed_subvillage','longitude_imputed_ward' , 'longitude_imputed_lga' , 'longitude_imputed_region', 'longitude'], axis=1)

In [454]:
# train_df['latitude'].where(train_df["latitude"] <= -0.5, np.nan, inplace=True)
# #create mean latitude on the lowest granularity level (subvillage)
# means_latitude_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['latitude'].mean().reset_index()
# means_latitude_subvillage = means_latitude_subvillage.rename(columns={"latitude": "latitude_imputed_subvillage"})

# #ward level
# means_latitude_ward = train_df.groupby(['region', 'lga', 'ward',])['latitude'].mean().reset_index()
# means_latitude_ward = means_latitude_ward.rename(columns={"latitude": "latitude_imputed_ward"})

# #lga level
# means_latitude_lga = train_df.groupby(['region', 'lga'])['latitude'].mean().reset_index()
# means_latitude_lga = means_latitude_lga .rename(columns={"latitude": "latitude_imputed_lga"})

# #region level
# means_latitude_region = train_df.groupby(['region'])['latitude'].mean().reset_index()
# means_latitude_region = means_latitude_region.rename(columns={"latitude": "latitude_imputed_region"})
# means_latitude_region.head()

In [455]:
# #merge the aggregated dataframes as new columns to the original df
# train_df= train_df.merge(means_latitude_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
# train_df= train_df.merge(means_latitude_ward, how = 'left', on = ['region', 'lga', 'ward'])
# train_df = train_df.merge(means_latitude_lga, how = 'left', on = ['region', 'lga'])
# train_df = train_df.merge(means_latitude_region, how = 'left', on = ['region'])

# #select the right latitude level based on the availability of information
# train_df['imputed_latitude'] = np.where(train_df['latitude'].isna(), train_df['latitude_imputed_subvillage'], train_df['latitude']) #if longitude is missing, impute it by the mean of the subvillage
# train_df['imputed_latitude'] = np.where(train_df['imputed_latitude'].isna(), train_df['latitude_imputed_ward'], train_df['imputed_latitude']) #if subvillage mean is missing, impute it by the ward
# train_df['imputed_latitude'] = np.where(train_df['imputed_latitude'].isna(), train_df['latitude_imputed_lga'], train_df['imputed_latitude'])
# train_df['imputed_latitude'] = np.where(train_df['imputed_latitude'].isna(), train_df['latitude_imputed_region'], train_df['imputed_latitude'])

# #drop redundant columns
# train_df= train_df.drop(['latitude_imputed_subvillage','latitude_imputed_ward' , 'latitude_imputed_lga' , 'latitude_imputed_region', 'latitude'], axis=1)



### Permit Imputation
Impute missing values with mode of similar records. `management_group` and `public_meeting` were used by BrendaLoznik.

In [456]:
#impute by mode
permit_mg_mode= train_df.groupby(['public_meeting', 'management_group'])['permit'].agg(pd.Series.mode).reset_index()
permit_mg_mode  = permit_mg_mode.rename(columns={"permit": "imputed_permit_mg"})
train_df = train_df.merge(permit_mg_mode, how = 'left', on = ['public_meeting', 'management_group'])

train_df['imputed_permit'] = np.where(train_df['permit'].isna(), train_df['imputed_permit_mg'], train_df['permit'])  #if permit is missing, replace it by the mode of public meeting - management group
train_df['imputed_permit']  = np.where(train_df['imputed_permit'] .isna(), train_df['permit'].mode(), train_df['imputed_permit'])  #if eitther public meeting or management group is missing, then use the mode of permit (True)

#drop original permit column
train_df = train_df.drop(['permit', 'imputed_permit_mg'], axis=1)

### Public Meeting Imputation
Simply impute public_meeting with the mode, as 90% of pumps have a TRUE value

Solution by BrendaLoznik

In [457]:
public_meeting_mode = train_df['public_meeting'].mode()[0]
train_df['public_meeting'] = train_df['public_meeting'].fillna(public_meeting_mode)

### Scheme Management Imputation

Solution by BrendaLoznik

In [458]:
# it seems like a logical choice to impute missing scheme management values by the mode of the management - scheme-management as there is a lot of overlap here
check = train_df.groupby([  'management_group', 'scheme_management' , 'management'])['id'].count().reset_index()
check = check.sort_values('management')
check.head(10)

scheme_mode = train_df.groupby(['management'])['scheme_management'].agg(pd.Series.mode).reset_index()
scheme_mode = scheme_mode.rename(columns={"scheme_management": "imputed_scheme_management"})
scheme_mode 

#merge scheme_mode to original df and use it to replace missing values
train_df = train_df.merge(scheme_mode, how = 'left', on = [ 'management'])
train_df['imputed_scheme__management'] = np.where(train_df['scheme_management'].isna(), train_df['imputed_scheme_management'], train_df['scheme_management'])

#drop redundant columns
train_df= train_df.drop(['scheme_management', 'imputed_scheme_management'],axis=1)



### Installer

Solution by BrendaLoznik

In [459]:
#I noticed a lot of variation in captialization, so I will first convert al entries to lowercase
train_df['installer'] = train_df['installer'].str.lower()

#plot top 10 installers
installer = train_df.groupby('installer')['id'].count().reset_index()
installer = installer.sort_values('id', ascending = False)
installer.head(10)


#there are a few categories in the top 60 most common categories whose naims look a lot alike and are probably typo's. We will merge them together
train_df['installer'] = np.where( train_df['installer']=='gove', 'gover', train_df['installer'] )
train_df['installer'] = np.where( train_df['installer']=='community', 'commu', train_df['installer'] )
train_df['installer'] = np.where( train_df['installer']=='danid', 'danida', train_df['installer'] )

installer = train_df.groupby('installer')['id'].count().reset_index()
installer = installer.sort_values('id', ascending = False)
installer.head(10)

#I want to keep the most frequent installers and combine the rarer classes together. I have played around with the optimum number of installers to keep, and I decided on the top 150.

#create list of top 150 installers
top_installers = installer.nlargest(150, 'id')['installer'].unique()

#replace funders that are not in top 10 with 'other'
train_df['installer'] = np.where(train_df['installer'].isin(top_installers), train_df['installer'], 'other')


### Funder

Solution by BrendaLoznik

In [460]:
#set all entries to lowercase
train_df['funder'] = train_df['funder'].str.lower()


funder = train_df.groupby('funder')['id'].count().reset_index()
funder = funder.sort_values('id', ascending = False)
funder.head(10)

#create list of top 150 funders
top_funders = funder.nlargest(150, 'id')['funder'].unique()

#replace funders that are not in top 150 with 'other'
train_df['funder'] = np.where(train_df['funder'].isin(top_funders), train_df['funder'], 'other')


### Construction Year Imputation

Since `extraction_type_group` is associated with `construction_year`, that will help to impute the values.

Solution by BrendaLoznik

In [461]:
#because of the wide spread of construction years, I don't want to impude by the overall mean
train_df['construction_year'].min(),  train_df['construction_year'].max() , train_df['construction_year'].mean()

#We can see that the mean construction year by extraction type group gives much more detailed information
mean_construction = train_df.groupby('extraction_type_group')['construction_year'].mean().reset_index()

#create train_df with the mean extraction year by extraction type group
mean_construction = train_df.groupby('extraction_type_group')['construction_year'].mean().reset_index()
mean_construction  = mean_construction .rename(columns={"construction_year": "imputed_construction_year"})

#merge this train_df to the main train_df and replace missing values
train_df = train_df.merge(mean_construction, how =  'left', on =  'extraction_type_group')
train_df['construction_year_imputed'] = np.where(train_df['construction_year'].isna(), train_df['imputed_construction_year'], train_df['construction_year'] )

#drop redundant columns
train_df=train_df.drop(['imputed_construction_year', 'construction_year'], axis=1)

### GPS Height Imputation

Replace values randomly within 1std of the mean, following a normal distribution.

This probably needs verifying/improving but it'll do for now.

Solution by BrendaLoznik

In [462]:
#create mean on the lowest granularity level (subvillage)
means_altitude_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['gps_height'].mean().reset_index()#
means_altitude_subvillage = means_altitude_subvillage.rename(columns={"gps_height": "gps_height_imputed_subvillage"})

#ward level
means_altitude_ward = train_df.groupby(['region', 'lga', 'ward',])['gps_height'].mean().reset_index()
means_altitude_ward = means_altitude_ward.rename(columns={"gps_height": "gps_height_imputed_ward"})

#lga level
means_altitude_lga = train_df.groupby(['region', 'lga'])['gps_height'].mean().reset_index()
means_altitude_lga = means_altitude_lga .rename(columns={"gps_height": "gps_height_imputed_lga"})

#region level
means_altitude_region = train_df.groupby(['region'])['gps_height'].mean().reset_index()
means_altitude_region = means_altitude_region.rename(columns={"gps_height": "gps_height_imputed_region"})

#region basin
means_altitude_basin = train_df.groupby(['basin'])['gps_height'].mean().reset_index()
means_altitude_basin = means_altitude_basin.rename(columns={"gps_height": "gps_height_imputed_basin"})

#merge the aggregated dataframes as new columns to the original train_df
train_df= train_df.merge(means_altitude_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
train_df = train_df.merge(means_altitude_ward, how = 'left', on = ['region', 'lga', 'ward'])
train_df = train_df.merge(means_altitude_lga, how = 'left', on = ['region', 'lga'])
train_df = train_df.merge(means_altitude_region, how = 'left', on = ['region'])
train_df = train_df.merge(means_altitude_basin, how = 'left', on = ['basin'])

#create final imputed longitude column
train_df['imputed_gps_height'] = np.where(train_df['gps_height'].isna(), train_df['gps_height_imputed_subvillage'], train_df['gps_height']) #if longitude is missing, impute it by the mean of the subvillage
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_ward'], train_df['imputed_gps_height']) #if subvillage mean is missing, impute it by the ward
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_lga'], train_df['imputed_gps_height'])
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_region'], train_df['imputed_gps_height'])
train_df['imputed_gps_height'] = np.where(train_df['imputed_gps_height'].isna(), train_df['gps_height_imputed_basin'], train_df['imputed_gps_height'])

#drop redundant columns
train_df= train_df.drop(['gps_height_imputed_subvillage','gps_height_imputed_ward' , 'gps_height_imputed_lga' , 'gps_height_imputed_region', 'gps_height', 'gps_height_imputed_basin'], axis=1)


### Population
Brenda's solution considers region-wide population average when not available, which is a poor indicator of population. Some readings also have very large population values, which skew the average. We use a binning approach to better represent population, with a "missing" bin where subvillage or ward population is unknown, as this itself is a decent predictor.

In [463]:
#create mean on the lowest granularity level (subvillage)
means_population_subvillage = train_df.groupby(['region', 'lga', 'ward', 'subvillage'])['population'].mean().reset_index()#
means_population_subvillage = means_population_subvillage.rename(columns={"population": "population_imputed_subvillage"})

#ward level
means_population_ward = train_df.groupby(['region', 'lga', 'ward',])['population'].mean().reset_index()
means_population_ward = means_population_ward.rename(columns={"population": "population_imputed_ward"})

#lga level
means_population_lga = train_df.groupby(['region', 'lga'])['population'].mean().reset_index()
means_population_lga = means_population_lga .rename(columns={"population": "population_imputed_lga"})

#region level
means_population_region = train_df.groupby(['region'])['population'].mean().reset_index()
means_population_region = means_population_region.rename(columns={"population": "population_imputed_region"})

#region basin
means_population_basin = train_df.groupby(['basin'])['population'].mean().reset_index()
means_population_basin = means_population_basin.rename(columns={"population": "population_imputed_basin"})

#merge the aggregated dataframes as new columns to the original df
train_df= train_df.merge(means_population_subvillage, how = 'left', on = ['region', 'lga', 'ward', 'subvillage'])
train_df = train_df.merge(means_population_ward, how = 'left', on = ['region', 'lga', 'ward'])
train_df = train_df.merge(means_population_lga, how = 'left', on = ['region', 'lga'])
train_df = train_df.merge(means_population_region, how = 'left', on = ['region'])
train_df = train_df.merge(means_population_basin, how = 'left', on = ['basin'])

#create final imputed longitude column
train_df['imputed_population'] = np.where(train_df['population'].isna(), train_df['population_imputed_subvillage'], train_df['population']) #if longitude is missing, impute it by the mean of the subvillage
train_df['imputed_population'] = np.where(train_df['imputed_population'].isna(), train_df['population_imputed_ward'], train_df['imputed_population']) #if subvillage mean is missing, impute it by the ward
train_df['imputed_population'] = np.where(train_df['imputed_population'].isna(), train_df['population_imputed_lga'], train_df['imputed_population'])
train_df['imputed_population'] = np.where(train_df['imputed_population'].isna(), train_df['population_imputed_region'], train_df['imputed_population'])
train_df['imputed_population'] = np.where(train_df['imputed_population'].isna(), train_df['population_imputed_basin'], train_df['imputed_population'])

#drop redundant columns
train_df= train_df.drop(['population_imputed_subvillage','population_imputed_ward' , 'population_imputed_lga' , 'population_imputed_region', 'population', 'population_imputed_basin'], axis=1)

### Correct data types

In [464]:
#replace string to integer
train_df['public_meeting'] = train_df['public_meeting'].replace({True: 1, False: 0})
train_df['imputed_permit'] = train_df['imputed_permit'].replace({True: 1, False: 0})


#change to integer
train_df[['imputed_gps_height', 'construction_year_imputed', 'imputed_population']] = train_df[['imputed_gps_height', 'construction_year_imputed', 'imputed_population']].astype('int')

#change type to categorical
train_df[['region_code', 'district_code', 'num_private']] = train_df[['region_code', 'district_code', 'num_private']].astype('str')
train_df[[ 'num_private']] = train_df[[ 'num_private']].astype('str')

#remove decimal
train_df['district_code'] = train_df['district_code'].str.split(".").str[0]


In [465]:
#save cleaned df
train_df= train_df.rename(columns={"imputed_permit": "permit", "imputed_scheme__management": "scheme_management",
                    "imputed_gps_height": "gps_height", 'construction_year_imputed': 'construction_year', 
                   'imputed_population': 'population', 'imputed_longitude': 'longitude'}, errors="raise")
# , 'imputed_latitude': 'latitude'

train_df.to_csv("cleaned_df.csv")

## 2. Feature Engineering

### recorded_age Feature
Create a feature recorded_age, which states the age (in years) of the pump at time of recording. Uses date_recorded and construction_year.

In [466]:
#create age feature
train_df['recorded_year'] = pd.DatetimeIndex(train_df ['date_recorded']).year
train_df[ 'age'] = train_df['recorded_year'] - train_df['construction_year']
train_df = train_df.drop('recorded_year',axis=1)

### Recorded Season
The month in which the water pump's functionality is recorded may be important due to wet/dry seasons in Tanzania. Due to the poor distribution of recorded months, we group months into different season bins.

In [467]:
train_df['month'] = pd.DatetimeIndex(train_df['date_recorded']).month

# season encoder
season_mapper = {1: 'short dry',2:'short dry', 3: 'long rain', 4: 'long rain', 5: 'long rain',6: 'long dry', 
                 7: 'long dry', 8: 'long dry', 9: 'long dry', 10: 'long dry', 11:'short rain', 12:'short rain'}
#.p feature values to scale
train_df['season'] = train_df['month'].replace(season_mapper)
train_df = train_df.drop('month', axis=1)

### amount_tsh missing
Use a binary feature for whether amount_tsh is missing - this is a decent predictor of water pump status

In [468]:
train_df['amount_tsh_missing'] = np.where( train_df['amount_tsh'].isna(), 1, 0)

### Region District

In [469]:
train_df['region_district'] = train_df['region']+ "-" + train_df['district_code']

### Reduce Cardinality of Longitude/Latitude

In [470]:
#two decimal places is 1.1 km accurate. This will provide enough information on the location. Using the full coordinate doesn't provide a lot of general information, but does result in high cardinality
train_df['longitude'] = round(train_df['longitude'], 2)
train_df['latitude'] = round(train_df['latitude'],2)

### Extraction

In [471]:
#i want to keep extraction type class and I will group the extraction type group en type together

#swn 80 and swn 81 become swn
#cemo + climax become other motorpump
# other -mkulima, other -play and walimi become other handpump

swn = ['other - swn 81', 'swn80']
train_df['extraction_type'] =np.where(train_df['extraction_type'].isin(swn), 'swn',  train_df['extraction_type'])

other_handpump = ['other - mkulima/shinyanga', 'other - play pump', 'other - walimi']
train_df['extraction_type']=np.where(train_df['extraction_type'].isin(other_handpump), 'other handpump',  train_df['extraction_type'])

other_motorpump = ['cemo', 'climax']
train_df['extraction_type'] =np.where(train_df['extraction_type'].isin(other_motorpump), 'other motorpump',  train_df['extraction_type'])


### Autonomy

In [472]:
#autonomous = government, VWC, town council ..... also water authority?, parastatal (=state company)? SWC?
#autonomous = WUA, WUG, board, trust, school
#private = private, company

non = ['VWC', 'Water authority', 'Parastatal', 'SWC']
autonomous = ['WUG', 'WUA', 'Water Board', 'Trust']
private = ['Company', 'Private operator']
other = ['None', 'Other']

train_df['authority_scheme'] = train_df['scheme_management']
train_df.loc[train_df['authority_scheme'].isin(non),'authority_scheme']='non-autonomous'
train_df.loc[train_df['authority_scheme'].isin(autonomous ),'authority_scheme']='autonomous'
train_df.loc[train_df['authority_scheme'].isin(private),'authority_scheme']='private'
train_df.loc[train_df['authority_scheme'].isin(other ),'authority_scheme']='other'

### Combine rare features in `source`

In [473]:
#keep source, but the rare classes will be put together
other = ['other',  'unknown']
train_df['source'] = np.where(train_df['source']=='unknown', 'other', train_df['source'])

## 3. (Partial) Feature Selection
Remove bad features here.

### Columns to Drop
amount_tsh: *70% missing values*

date_recorded: *Used in recorded_age engineered feature*



In [474]:
#during EDA I already decided what features to keep and which ones to drop
train_df = train_df.drop(['amount_tsh', 'date_recorded', 'wpt_name', 'num_private', 'subvillage', 'region',  'district_code', 'lga', 'ward', 'recorded_by', 'scheme_name', 'extraction_type_group', 'management', 'management_group', 'payment', 'quality_group', 'quantity_group', 'source_class', 'source_type', 'waterpoint_type_group', 'construction_year'], axis=1)
train_df.head()

,id,status_group,funder,installer,latitude,basin,region_code,public_meeting,extraction_type,extraction_type_class,payment_type,water_quality,quantity,source,waterpoint_type,longitude,permit,scheme_management,gps_height,population,age,season,amount_tsh_missing,region_district,authority_scheme
0,69572,functional,roman,roman,-9.86,Lake Nyasa,11,1,gravity,gravity,annually,soft,enough,spring,communal standpipe,34.94,0,VWC,1390,109,12,long rain,0,Iringa-5,non-autonomous
1,8776,functional,grumeti,grumeti,-2.15,Lake Victoria,20,1,gravity,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe,34.70,1,Other,1399,280,3,long rain,1,Mara-2,other
2,34310,functional,other,world vision,-3.82,Pangani,21,1,gravity,gravity,per bucket,soft,enough,dam,communal standpipe multiple,37.46,1,VWC,686,250,4,short dry,0,Manyara-4,non-autonomous
3,67743,non functional,unicef,unicef,-11.16,Ruvuma / Southern Coast,90,1,submersible,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple,38.49,1,VWC,263,58,27,short dry,1,Mtwara-63,non-autonomous
4,19728,functional,other,artisan,-1.83,Lake Victoria,18,1,gravity,gravity,never pay,soft,seasonal,rainwater harvesting,communal standpipe,31.13,1,Other,1328,532,15,long dry,1,Kagera-1,other


## 4. Encoding
Remove bad features here.

In [475]:
#define train and test 
train_set = train_df[train_df ["id"].isin(train_base_df["id"])]
test_set =  train_df[train_df  ["id"].isin(test_df["id"])]

In [476]:
X = train_set.drop(['id', 'status_group'], axis=1)
y = train_set['status_group']

In [477]:
X.columns

Index(['funder', 'installer', 'latitude', 'basin', 'region_code',
       'public_meeting', 'extraction_type', 'extraction_type_class',
       'payment_type', 'water_quality', 'quantity', 'source',
       'waterpoint_type', 'longitude', 'permit', 'scheme_management',
       'gps_height', 'population', 'age', 'season', 'amount_tsh_missing',
       'region_district', 'authority_scheme'],
      dtype='object')

In [478]:
categorical_cols = [cname for cname in X .columns if
                   X [cname].dtype == "object"]

In [479]:
# Encode Categorical Columns 
for col in categorical_cols:
  le = LabelEncoder()
  train_set[col] = le.fit_transform(train_set[col])
  test_set[col] = le.transform(test_set[col])


/var/folders/vg/953jt4190_x2bp1n7tcvzfpr0000gn/T/ipykernel_50619/3744793663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[col] = le.fit_transform(train_set[col])
/var/folders/vg/953jt4190_x2bp1n7tcvzfpr0000gn/T/ipykernel_50619/3744793663.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set[col] = le.transform(test_set[col])
/var/folders/vg/953jt4190_x2bp1n7tcvzfpr0000gn/T/ipykernel_50619/3744793663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [480]:
#merge the encoded train and test sets together
train_df = train_set.append(test_set)

/var/folders/vg/953jt4190_x2bp1n7tcvzfpr0000gn/T/ipykernel_50619/3238694098.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_set.append(test_set)


In [481]:
train_df.to_csv("final1_df.csv")

In [482]:
train_df.columns

Index(['id', 'status_group', 'funder', 'installer', 'latitude', 'basin',
       'region_code', 'public_meeting', 'extraction_type',
       'extraction_type_class', 'payment_type', 'water_quality', 'quantity',
       'source', 'waterpoint_type', 'longitude', 'permit', 'scheme_management',
       'gps_height', 'population', 'age', 'season', 'amount_tsh_missing',
       'region_district', 'authority_scheme'],
      dtype='object')